## Imports

In [5]:
%load_ext autoreload
%autoreload 2

In [47]:
import torch
import numpy as np
from transformers import GPTNeoXForCausalLM, AutoTokenizer
from safetensors.torch import load_file
from bergson.data import IndexConfig
from bergson.approx_unrolling.utils import TensorDict
import json
import os
from dataclasses import asdict
from datetime import timedelta

import torch
import torch.distributed as dist
from datasets import Dataset, IterableDataset
from torch.distributed.fsdp import fully_shard
from tqdm.auto import tqdm
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, PreTrainedModel
from datasets import Dataset, DatasetDict, IterableDatasetDict, load_dataset, load_from_disk

from bergson.data import IndexConfig, allocate_batches, DataConfig
from bergson.distributed import distributed_computing
from bergson.gradients import GradientProcessor
from bergson.data import IndexConfig, tokenize

from bergson.utils import assert_type, get_layer_list
import os

## 0. Hyperparameters

In [39]:
cfg = IndexConfig(run_path="")  # empty run path because we are not using it to save data
cfg.model = "EleutherAI/Pythia-14m"
cfg.precision = "fp16"
cfg.revision = None
cfg.fsdp = False
cfg.normalizer = "none"
cfg.fisher_fourth_root = False
cfg.data = DataConfig(dataset="/root/bergson-approx-unrolling/tests/ekfac_tests/test_files/pile_8_entries")
cfg.data= Dataconfig
data_str = cfg.data.dataset


In [40]:
match cfg.precision:
    case "bf16":
        dtype = torch.bfloat16
    case "fp16":
        dtype = torch.float16
    case "fp32":
        dtype = torch.float32
    case "int4" | "int8":
        dtype = torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16
    case other:
        raise ValueError(f"Unsupported precision: {other}")


In [41]:
rank = 0

## 1. Loading model and data

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    cfg.model,
    device_map="cuda",
    quantization_config=(
        BitsAndBytesConfig(
            load_in_4bit=cfg.precision == "int4",
            load_in_8bit=cfg.precision == "int8",
            bnb_4bit_compute_dtype=dtype,
            bnb_4bit_quant_storage=dtype,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_use_double_quant=True,
        )
        if cfg.precision in ("int4", "int8")
        else None
    ),
    torch_dtype=dtype,
    revision=cfg.revision,
)


In [45]:
tokenizer = AutoTokenizer.from_pretrained(cfg.model, model_max_length=cfg.token_batch_size, revision=cfg.revision)


tokenizer_config.json:   0%|          | 0.00/264 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

In [61]:
ds = load_from_disk(data_str)
remove_columns = ds.column_names if cfg.drop_columns else None


ds = ds.map(
    tokenize,
    batched=True,
    fn_kwargs=dict(args=cfg.data, tokenizer=tokenizer),
    remove_columns=remove_columns,
)

In [63]:
ds["length"]

[3146, 1236, 265, 81, 385, 515, 848, 229]

In [65]:
batches = allocate_batches(doc_lengths=ds["length"], N=cfg.token_batch_size, workers=8)

[[0, 1], [6, 5, 4, 2, 7, 3]]
[(3146, 2), (848, 6)]


RuntimeError: Not enough documents to give each worker at least one batch.

In [56]:
batches

[[0, 1], [6, 5, 4, 2, 7, 3]]

In [50]:
target_modules = None
normalizers = {}

processor = GradientProcessor(
    normalizers=normalizers,
    fisher_fourth_root=cfg.fisher_fourth_root,
    projection_dim=None,
)
